In [86]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL, lang='english', check=False)

from absbox.examples import trigger01,trigger02

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.26.4, server:0.27.8

##### Trigger 01

the test case `trigger01` which is in `PreClosing` stage ,has a trigger with constant default rate. When the cumulative default rate breached, the deal status will convert into `Accelerated`

    defautlRateTrigger = {"defaultRateTrigger": 
                            {"condition":[("cumPoolDefaultedRate",),">", 0.05]
                            ,"effects":("newStatus","Accelerated")
                            ,"status":False
                            ,"curable":False}}

In [39]:
r = localAPI.run(trigger01
                 ,poolAssump = ("Pool",("Mortgage",{"CDR":0.08},None,None,None)
                                       ,None
                                       ,None)
                 ,runAssump = [("inspect",("MonthEnd",("trigger", "AfterCollect" ,"defaultRateTrigger"))
                                         ,("MonthEnd",("cumPoolDefaultedBalance",)))
                               ]
                 ,read=True)

Warning Message from server:
Bond B is not paid off
Account acc01 has cash to be distributed
No waterfall distribution found on date2021-12-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-01-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-02-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-03-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-04-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-05-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-06-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-07-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-08-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-09-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-10-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-11-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-12-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-01-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-02-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-03-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-04-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-05-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-06-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-07-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-08-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-09-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-10-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-11-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-12-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-01-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-02-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-03-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-04-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-05-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-06-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-07-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-08-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-09-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-10-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-11-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-12-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-01-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-02-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-03-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-04-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-05-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-06-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-07-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-08-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-09-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025

##### Why so many warnings ? 

Because the testing case hasn't specify the waterfall for `Accelerated` status.

In [53]:
cmulativeDefaultRates = r['pool']['flow']['CumDefault'] / 2200
cmulativeDefaultRates.iloc[6:10,]

Date
2021-10-01    0.044818
2021-11-01    0.048927
2021-12-01    0.052559
2022-01-01    0.055959
Name: CumDefault, dtype: float64

In [54]:
triggerStatus = r['result']['inspect']['<TriggersStatus:EndCollectionWF,defaultRateTrigger>']
triggerStatus.iloc[8:12,]

,"<TriggersStatus:EndCollectionWF,defaultRateTrigger>"
Date,
2021-10-31,False
2021-11-30,False
2021-12-31,True
2022-01-31,True


In [55]:
r['result']['inspect']['<CumulativePoolDefaultedBalance>'].iloc[8:12,]

,<CumulativePoolDefaultedBalance>
Date,
2021-10-31,98.6
2021-11-30,107.64
2021-12-31,115.63
2022-01-31,123.11


In [56]:
pd.concat([triggerStatus, cmulativeDefaultRates], axis=1).sort_index().iloc[15:22,]

,"<TriggersStatus:EndCollectionWF,defaultRateTrigger>",CumDefault
Date,,
2021-10-31,False,NaN
2021-11-01,NaN,0.048927
2021-11-30,False,NaN
2021-12-01,NaN,0.052559
2021-12-31,True,NaN
2022-01-01,NaN,0.055959
2022-01-31,True,NaN


In [57]:
r['result']['status']

,Date,From,To
0,2021-04-01,PreClosing,Amortizing
1,2021-12-01,Amortizing,Accelerated


Now the trigger was fired at `2021-12-01` and deal status convert into `Accelerated`. After this date, deal will pick up waterfall which has a key `("amortizing", "accelerated")`

##### Trigger 02

the test case `trigger02` which is in `Amortizing` stage with `current defaulted balance` = 100 ,has a same trigger as the one shown in `trigger01`.

###### Note

`trigger02` is ongoing deal which is likely accumulates some defaults. Here , you may notice `issuanceStat` has a defaulted balance of 100.

In [73]:
trigger02.pool

{'assets': [['Mortgage',
   {'originBalance': 2200,
    'originRate': ['fix', 0.045],
    'originTerm': 20,
    'freq': 'Monthly',
    'type': 'Level',
    'originDate': '2021-02-01'},
   {'currentBalance': 2200,
    'currentRate': 0.08,
    'remainTerm': 20,
    'status': 'current'}]],
 'issuanceStat': {'HistoryDefaults': 50, 'IssuanceBalance': 1500}}

In [74]:
trigger02.dates

{'collect': ['2021-10-01', '2021-11-01'],
 'pay': ['2021-09-20', '2021-10-20'],
 'payFreq': ['DayOfMonth', 20],
 'poolFreq': 'MonthFirst',
 'stated': '2030-01-01'}

In [93]:
r = localAPI.run(trigger02
                 ,poolAssump = ("Pool",("Mortgage",{"CDR":0.03},None,None,None)
                                       ,None
                                       ,None)
                 ,runAssump = [("inspect",("MonthEnd",("trigger", "AfterCollect" ,"defaultRateTrigger"))
                                         ,("MonthEnd",("cumPoolDefaultedBalance",)))
                               ]
                 ,read=True)

Warning Message from server:
Bond B is not paid off
Account acc01 has cash to be distributed
No waterfall distribution found on date2022-09-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-10-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-11-20with statusDealAccelerated Nothing
No waterfall distribution found on date2022-12-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-01-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-02-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-03-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-04-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-05-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-06-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-07-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-08-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-09-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-10-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-11-20with statusDealAccelerated Nothing
No waterfall distribution found on date2023-12-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-01-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-02-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-03-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-04-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-05-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-06-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-07-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-08-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-09-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-10-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-11-20with statusDealAccelerated Nothing
No waterfall distribution found on date2024-12-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-01-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-02-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-03-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-04-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-05-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-06-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-07-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-08-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-09-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-10-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-11-20with statusDealAccelerated Nothing
No waterfall distribution found on date2025-12-20with statusDealAccelerated Nothing
No waterfall distribution found on date2026-01-20with statusDealAccelerated Nothing
No waterfall distribution found on date2026-02-20with statusDealAccelerated Nothing
No waterfall distribution found on date2026-03-20with statusDealAccelerated Nothing
No waterfall distribution found on date2026-04-20with statusDealAccelerated Nothing
No waterfall distribution found on date2026-05-20with statusDealAccelerated Nothing
No waterfall distribution found on date2026-06-20with statusDealAccelerated Nothing
No waterfall distribution found on date2026

In [94]:
triggerStatus = r['result']['inspect']['<TriggersStatus:EndCollectionWF,defaultRateTrigger>']
cumulativeDefaultAmt = r['result']['inspect']['<CumulativePoolDefaultedBalance>']
#cumulativeDefaultAmt / 1500

In [95]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.concat([triggerStatus, cumulativeDefaultAmt / 1500], axis=1).sort_index().iloc[10:15,]

,"<TriggersStatus:EndCollectionWF,defaultRateTrigger>",<CumulativePoolDefaultedBalance>
Date,,
2022-07-31,False,0.049173
2022-08-31,False,0.049747
2022-09-30,True,0.050127
2022-10-31,True,0.050307
2022-11-30,True,0.050307


In [96]:
r['result']['status']

,Date,From,To
0,2022-09-01,Amortizing,Accelerated
